# Fedbiomed Researcher to train a model on a CSV dataset

Use for developing (autoreloads changes made across packages)

In [ ]:
%load_ext autoreload
%autoreload 2

This example shows how to use a CSV format file as a node dataset. The example CSV file is synthetic data with a format inspired from ADNI dataset.

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 1 to add a csv file to the node
  * Choose the name, tags and description of the dataset
    * use `#test_data` for the tags
  * Pick the .csv file from your PC (here: [pseudo_adni_mod.csv](./data/CSV/pseudo_adni_mod.csv))
  * Data must have been added
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node start`. Wait until you get `Starting task manager`. it means you are online.

## Create an experiment to train a model on the data found

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torch.utils.data import Dataset
import pandas as pd

# Here we define the model to be used. 
# You can use any class name (here 'MyTrainingPlan')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        # should match the model arguments dict passed below to the experiment class
        self.in_features = model_args['in_features']
        self.out_features = model_args['out_features']
        self.fc1 = nn.Linear(self.in_features, 5)
        self.fc2 = nn.Linear(5, self.out_features)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch Dataset and DataLoader classes
        # We need pandas to read the local .csv file at the node side
        deps = ["from torch.utils.data import Dataset, DataLoader",
                "import pandas as pd"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    def training_step(self, data, target):
        output = self.forward(data).float()
        criterion = torch.nn.MSELoss()
        loss   = criterion(output, target.unsqueeze(1))
        return loss

      # OLD WAY 
#     class csv_Dataset(Dataset):
#     # Here we define a custom Dataset class inherited from the general torch Dataset class
#     # This class takes as argument a .csv file path and creates a torch Dataset 
#         def __init__(self, dataset_path, x_dim):
#             self.input_file = pd.read_csv(dataset_path,sep=';',index_col=False)
#             x_train = self.input_file.iloc[:,:x_dim].values
#             y_train = self.input_file.iloc[:,-1].values
#             self.X_train = torch.from_numpy(x_train).float()
#             self.Y_train = torch.from_numpy(y_train).float()

#         def __len__(self):            
#             return len(self.Y_train)

#         def __getitem__(self, idx):

#             return (self.X_train[idx], self.Y_train[idx])
        
    def training_data(self,  batch_size = 48):
        # OLD WAY
        # The training_data creates the Dataloader to be used for training in the 
        # general class TorchTrainingPlan of fedbiomed
        # dataset = self.csv_Dataset(self.dataset_path, self.in_features)
        
        # NEW WAY
        # Training data method reads the dataset and craete DataManager by passing 
        # input variables and output(target) variable. DataManager will automaticly 
        # create PyTorch Dataset instnce for training
        
        df = pd.read_csv(self.dataset_path, sep=';', index_col=False)
        x_dim = self.in_features
        x_train = df.iloc[:,:x_dim].values
        y_train = df.iloc[:,-1].values
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        
        data_manager = DataManager(dataset=x_train , target=y_train, **train_kwargs)
        
        return data_manager

In [2]:
# model parameters 
model_args = {
    'in_features': 15, 
    'out_features': 1
}

# training parameters 
training_args = {
    'batch_size': 20, 
    'lr': 1e-3, 
    'epochs': 10, 
    'dry_run': False,  
    #'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [3]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

# Calling the training data with specified tags
tags =  ['#test_data']
rounds = 5

exp = Experiment(tags=tags,
                 model_class=MyTrainingPlan,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-03-08 16:38:10,642 fedbiomed INFO - Component environment:
2022-03-08 16:38:10,643 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-08 16:38:10,817 fedbiomed INFO - Messaging researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f2da6538e50>
2022-03-08 16:38:10,828 fedbiomed INFO - Searching dataset with data tags: ['#test_data'] for all nodes
2022-03-08 16:38:10,830 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / DEBUG - Message received: {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'tags': ['#test_data'], 'command': 'search'}
2022-03-08 16:38:20,868 fedbiomed INFO - Node selected for training -> node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0
2022-03-08 16:38:20,905 fedbiomed DEBUG - Model file has been saved: /home/scansiz/Desktop/Inria/development/fedbiomed/var/experiments/Experiment_0096/my_model_3eb0c81a-136a-461b-bc4c-49d2858cd

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [4]:
exp.run()

2022-03-08 16:38:20,971 fedbiomed INFO - Sampled nodes in round 0 ['node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0']
2022-03-08 16:38:20,974 fedbiomed INFO - Send message to node node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 - {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'job_id': 'c851d69a-94c2-4e35-a885-7dfdaa54853c', 'training_args': {'batch_size': 20, 'lr': 0.001, 'epochs': 10, 'dry_run': False}, 'model_args': {'in_features': 15, 'out_features': 1}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/08/my_model_3eb0c81a-136a-461b-bc4c-49d2858cd9f2.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/08/aggregated_params_init_058e957b-3b11-4d07-aac8-fb36d3721312.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0': ['dataset_644ad54e-bf5f-45f7-a76c-7d88eaed4d73']}}
2022-03-08 16:38:20,975 fedbiomed DEBUG - researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab
2022-03-08 16:38:20,981 fedbiom

2022-03-08 16:38:21,622 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [0/1000 (0%)]	Loss: 233.559036
2022-03-08 16:38:21,632 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [200/1000 (20%)]	Loss: 155.664841
2022-03-08 16:38:21,644 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [400/1000 (40%)]	Loss: 336.358185
2022-03-08 16:38:21,655 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [600/1000 (60%)]	Loss: 374.487854
2022-03-08 16:38:21,665 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 7 [800/1000 (80%)]	Loss: 365.745087
2022-03-08 16:38:21,675 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO - Train Epoch: 8 [0/1000 (0%)]	Loss: 168.363968
2022-03-08 16:38:21,684 fedbiomed INFO - log from: node_bd90cc6f-67c4-4f81-aaa5-588c9c26c6e0 / INFO 


--------------------
Fed-BioMed researcher stopped due to keyboard interrupt
--------------------


Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available in `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())
